# IMPORT

In [1]:
import os, shutil #Controlling Files and paths and folder control
from shutil import copyfile
from pdf2image import convert_from_path #Libary convert pdf file to img file
from google.oauth2 import service_account #Control API Keys
from google.cloud import vision # Vision API from Google
import io
import string, random
import pandas as pd
# Imports the Google Cloud client library
from google.cloud import language_v1
from google.cloud.language_v1 import enums

In [2]:
#from __future__ import unicode_literals, print_function
#from spacy.lang.en import English # updated

# PDF TO TEXT

In [3]:
#Paths that have all folders which use everything
pdfIMGPopplerPath = '/Users/kunal/Documents/VdartResumeProject/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/'
imgTxtVisionAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
runningDocumentPath = '/Users/kunal/Documents/VdartResumeProject/runningDoc/'
sourceFolderResumesPath = '/Users/kunal/Documents/VdartResumeProject/50_resumes/'
excelFilesPath = '/Users/kunal/Documents/VdartResumeProject/ExcelFiles/'
nlpAutoAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-NLP_create_data.json"
jsonFolderPath = '/Users/kunal/Documents/VdartResumeProject/JSONLFILES/'

In [4]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image):
    #Using the convert_from_path function
    #Same name as pdf but converted to img
    #Watch out for poppler -- necceasary to function
    os.chdir(uploaded_image_path) # Change the working diretory to path that contains the PDF file
    file_name = str(uploaded_image).replace('.pdf','') # file name for png still going to get changed later
    pages = convert_from_path(uploaded_image, 200,poppler_path=pdfIMGPopplerPath) #function to change pdf to img
    pageNumCount = 1 #numbering for all the different images if pdf is multiple pages
    outputNames = []
    for page in pages:
        output_file = file_name+"_"+str(pageNumCount) + '.jpg'#uptaded name for image
        page.save(output_file, 'JPEG')#save img
        pageNumCount +=1
        outputNames.append(output_file)
    return outputNames #names of img

In [5]:
def convert_img_2_text(imagePath):
    #Using API from Google
    #Returns a JSON file but text is extracted from it
    keyDIR = imgTxtVisionAPIPath #JSON key file to call the api
    credentials = service_account.Credentials.from_service_account_file(keyDIR) #using service account to go through google
    client = vision.ImageAnnotatorClient(credentials=credentials) # client api
    with io.open(imagePath, 'rb') as image_file: #opening and reading img
        content = image_file.read() 
    image = vision.types.Image(content=content) # calling and running the client Visison API
    response = client.text_detection(image=image) # JSON return with all values
    texts = response.text_annotations # Go through Json and extract the text detected
    totalString = ''
    for text in texts:
        totalString+=text.description
    totalString = totalString.rsplit(' ', 1)[0] # throws away all the metadata with location -- could be used later
    return totalString #TEXT detected

In [6]:
def deleteIMG(path):
    #Deletes all the images from a folder
    for i in os.listdir(path):
        if i.endswith(".jpg"):
            os.remove(folder_pdf + i)
def deleteEverythingInFolder(folder_pdf):
    #deletes everything in the folder including folders and files
    for file in os.listdir(folder_pdf):
        try:
            shutil.rmtree(folder_pdf+ file) #remove folder
        except NotADirectoryError:
            try:
                os.remove(folder_pdf+ file) # if it is not a folder than it is a file so it removes it also
            except:
                pass
def moveRenameAllFiles(sourceFolderResumes, folder_pdf):
    #Moves all files from source and renames them starting from 100
    #If start from 1 then the ordering will be off
    for realFile in os.listdir(sourceFolderResumes):
        copyfile(sourceFolderResumes + realFile, folder_pdf + realFile) #duplicates all the files from source
    os.chdir(folder_pdf) # move them to the running folder
    documentcounter = 100 # names "Dociment_[num].pdf" start from 100 cause that is how numbering is listed correctly
    for i in os.listdir(folder_pdf):
        fileNameCreator = "Document_" + str(documentcounter) + ".pdf"
        print(i + "   changed to  " + fileNameCreator)
        os.rename(i, fileNameCreator)  # change name of file
        documentcounter+=1

In [7]:
folder_pdf = runningDocumentPath # set var
sourceFolderResumes = sourceFolderResumesPath

In [8]:
deleteEverythingInFolder(folder_pdf)
moveRenameAllFiles(sourceFolderResumes, folder_pdf)
numberOfFiles = len([name for name in os.listdir(folder_pdf) if os.path.isfile(name)]) 
#PNGFilesExist = checkifPNGExists(folder_pdf)
#deletes everything in the folder, copy and moves all files and renames them starting from 100

Document_402.pdf   changed to  Document_100.pdf
Document_403.pdf   changed to  Document_101.pdf
Document_405.pdf   changed to  Document_102.pdf
Document_406.pdf   changed to  Document_103.pdf
Document_407.pdf   changed to  Document_104.pdf
Document_408.pdf   changed to  Document_105.pdf
Document_409.pdf   changed to  Document_106.pdf
Document_410.pdf   changed to  Document_107.pdf
Document_411.pdf   changed to  Document_108.pdf
Document_412.pdf   changed to  Document_109.pdf
Document_413.pdf   changed to  Document_110.pdf
Document_414.pdf   changed to  Document_111.pdf
Document_415.pdf   changed to  Document_112.pdf
Document_417.pdf   changed to  Document_113.pdf
Document_418.pdf   changed to  Document_114.pdf
Document_419.pdf   changed to  Document_115.pdf
Document_420.pdf   changed to  Document_116.pdf
Document_421.pdf   changed to  Document_117.pdf
Document_422.pdf   changed to  Document_118.pdf
Document_423.pdf   changed to  Document_119.pdf
Document_424.pdf   changed to  Document_

In [9]:
#Asks the user which document they want to run and if they don't input anything, its a random document
doubleCheckDocumentInputCount = 0
while True:
    documentRun = input("Which Document do you wanna run?")
    if documentRun == "" and doubleCheckDocumentInputCount >= 1:
        documentRun = (random.choice(os.listdir(folder_pdf)))
        print("You choose RANDOM so your document is " + documentRun)
        break
    elif documentRun == "":
        print("This is a double check if you want a random document?")
        doubleCheckDocumentInputCount+=1
    elif documentRun == "BREAK":
        print("Exiting")
        documentRun = "INVALID"
        break
    else:
        documentRun = "Document_" + documentRun + ".pdf"
        print(os.path.isfile(folder_pdf + documentRun))
        if os.path.isfile(folder_pdf + documentRun):
            print("You choose a specific number so your document is " + documentRun)
            break
        else:
            print("Document was invalid. Input >>> BREAK <<< to stop")
            continue

Which Document do you wanna run?125
True
You choose a specific number so your document is Document_125.pdf


In [10]:
#Makes a folder and create images of each page in PDF document
textList = []
numberPagesList = []
for i in os.listdir(folder_pdf): 
    if i == documentRun:# only runs the file that was choosen by user
        os.chdir(folder_pdf) 
        folderNameCreate = folder_pdf+"Fold_"+i[:-4]+"/" 
        print(folderNameCreate)
        os.mkdir(folderNameCreate) #Create folder
        shutil.move(i,folderNameCreate) #MovePDF into folder
        os.chdir(folderNameCreate)
        for j in os.listdir(folderNameCreate):
            print(j)
            if j.endswith(".pdf"): 
                imgName = convert_pdf_2_image(folderNameCreate, j) #call pdf2img function
                print("Running Number: " + str(len(imgName))) 
                printString = ""
                for docName in imgName: # for making the printing better
                    printString += docName + " " 
                print(printString) 
            for q in range(len(imgName)):
                text = convert_img_2_text(folderNameCreate + i[:-4] + "_" + str(q+1) + ".jpg")
                textList.append(text)     #Convert to text and append all the text to list
                numberPagesList.append(len(imgName))   
    else:
        os.chdir(folder_pdf) #delete any other file that exists
        os.remove(i)

/Users/kunal/Documents/VdartResumeProject/runningDoc/Fold_Document_125/
Document_125.pdf
Running Number: 2
Document_125_1.jpg Document_125_2.jpg 


## Convert to Excel

In [11]:
df = pd.DataFrame()  # creates a tabel with all the text and pages
df['Text Extracted'] = textList
df['DocumentNum'] = numberPagesList

In [12]:
os.chdir(excelFilesPath)

In [13]:
EXCELFILENAME = documentRun[:-4] + "_text.xlsx"
df.to_excel(EXCELFILENAME, index = False) # save the text to the excel file folder

## Read Excel

In [14]:
EXCELFILENAME = documentRun[:-4] + "_text.xlsx" # read the same file

In [15]:
os.chdir(excelFilesPath) # change working path to excel files folder

In [16]:
text = pd.ExcelFile(EXCELFILENAME) 
dftext = text.parse("Sheet1") # read the file to see what was saved

In [17]:
dftext

,Text Extracted,DocumentNum
0,"Benjamin Meena\nDenver, CO 80202\nWeb Develope...",2
1,Managed a new and used vehicle inventory of gr...,2


In [18]:
text_list = dftext["Text Extracted"].tolist() # converts both columns to lists 
pagesList = dftext["DocumentNum"].tolist()

## Check for Recommendation letters

In [19]:
count=0
numb = 0
recNumList=[]
for i in text_list: # for all the text files scan if the word recommendation is in there
    if "Recommendation" in str(i): 
        recNumList.append(numb)
    numb+=1

In [20]:
recNumList # print all the page numbers with the word in it

[]

# Converts the text files to an array but still split into pages

In [21]:
runningTextFINAL = '\n'.join(textList) # the text is combined

# Create Sentences

In [22]:
print(runningTextFINAL) 

Benjamin Meena
Denver, CO 80202
Web Developer
Github: bmeena57.github.io
LinkedIn: linkedin.com/in/benjaminmeena
(757) 234-5481
ben.meena16@gmail.co
m
EXPERIENCE
Technical Highlights
HTML
- Certificate of Web
The Tech Academy, Denver
Development
CSS
JavaScript
AJAX
jquery
November 2019 - March 2020
Software Development
REACT
Bootstrap
Comprehensive overview of software development with courses in HTML5,
CSS3, JavaScript/jQuery, SQL/MS SQL Server, MVC, Source Control (Git),
and Agile product development methods (Scrum).
Non-Technical Highlights
Participated in a 2 week long live project with google meetings; updating
web pages for their client.
Self-driven
Strong Leadership
5 Guys Enterprise, Greenwood Village-Manager
November 2019- PRESENT
Management
Time efficient
Ensure that the Occupational Safety and Health Act, local health and safety codes,
and company safety and security policies are followed.
Coachable
Assist with recruiting, interviewing, and hiring of Crew Members
Sociable
Wo